In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(color_codes=True)
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv("data/test_df_full.csv")

data_XGB = pd.read_csv("data/test_data_full_XGB.csv")

data_XGB.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class,predicted_probability,prediction,FN,FP,proba_decile
0,-0.279746,-0.445189,1.236040,0.246604,0.173098,0.505621,-0.209898,-0.572866,-0.067674,0.001954,-0.104800,-0.094105,1.221842,0.463851,-0.457603,0.019923,0.490053,0.803178,-0.339824,0.385804,0.221326,-0.100578,-0.265297,-0.832876,0.083357,-0.060474,0.208230,0.099839,-0.031663,0.015881,0,0.001448,0,0,0,8
1,2.137218,-0.394131,1.106600,-0.983435,-0.435849,-0.965108,-0.195159,0.362881,-0.286150,-0.028419,-1.248617,0.678555,0.793663,0.297548,1.274832,-0.073223,0.203922,1.209132,-0.155889,-1.049597,0.830742,0.481581,0.325425,0.557488,-0.397485,-1.078475,0.657093,-0.052477,-0.019239,0.015687,0,0.001437,0,0,0,8
2,-0.279746,0.572081,2.053859,0.017620,-1.044302,0.398776,-0.017421,-1.103648,0.242713,-0.369792,0.378501,0.033398,-0.508773,1.025771,1.126608,0.068892,-0.029594,-0.213786,-0.292624,-1.089367,0.066974,-0.161975,-0.271011,-0.588222,0.333976,0.015987,-0.277432,0.197468,-0.060623,-0.058174,0,0.000236,0,0,0,2
3,-0.294977,-0.751195,-0.333837,1.250884,1.437889,0.263291,0.111567,-0.937753,0.611525,-0.152919,0.890815,-0.880124,0.732015,-2.586140,1.759959,1.004447,0.156358,0.354104,0.739067,0.135153,-0.291877,0.065634,-0.415962,-0.876026,-0.009245,0.268013,-0.183962,0.033665,0.219739,0.091659,0,0.005852,0,0,0,13
4,-0.238385,0.584582,-1.013271,1.867706,0.738315,2.500517,1.369829,0.188423,1.257136,-0.109642,-1.499215,1.349855,-1.262570,-0.164131,0.694228,-0.227402,-1.347315,0.677275,-0.981815,-0.752807,-1.470384,-0.054171,-0.238084,-0.539922,0.054420,0.668912,-0.531066,-0.536876,-0.475286,-0.239140,0,0.741086,1,0,1,19


In [3]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,scaled_amount,scaled_time,Class,predicted_probability,proba_decile,prediction,FN,FP
0,1.054379,-0.764756,0.160168,0.665587,1.113466,4.771656,-1.536609,1.306483,1.565054,-0.349941,-1.238496,0.608002,-0.249115,-0.933969,-1.407873,-0.191383,-0.180853,0.226980,0.537470,0.050995,-0.097138,-0.029759,-0.171596,1.047201,0.698595,-0.217939,0.101970,0.040152,0.478654,-0.182124,0,0.00,0,0,0,0
1,-4.805134,4.351191,-0.916135,-0.900752,-0.870777,-0.782946,0.328104,0.582501,2.749336,4.728060,0.455162,0.649740,0.930379,-1.663325,1.334738,-0.283321,0.152340,-0.950771,-0.935567,2.160193,-0.854486,-0.758266,0.147005,-0.021274,0.552660,0.008318,0.431835,-0.436239,-0.169378,-0.046108,0,0.01,17,0,0,0
2,-1.549833,-0.261143,1.556289,-2.037817,-0.366315,-1.334314,0.165406,-0.025782,-1.472751,-0.172838,-0.594842,0.063059,1.054185,-0.171535,0.192041,-0.867483,-0.348274,0.743352,-1.454672,-0.347365,-0.405656,-0.951005,-0.147752,0.424100,0.081845,1.103559,-0.502271,-0.248822,0.334863,-0.511327,0,0.01,17,0,0,0
3,0.216344,0.663182,1.303520,0.169219,-0.404266,-0.517288,-0.036938,-0.838221,-0.528675,-0.174777,1.648466,1.114947,0.202150,0.479424,0.226420,0.456003,-0.558258,-0.101939,0.000758,-0.184929,0.552788,-0.961206,-0.014642,0.525962,0.750877,0.141543,0.045107,0.127321,-0.256154,-0.987464,0,0.00,0,0,0,0
4,-0.273365,0.825649,0.555674,0.384915,1.343842,1.984159,0.321808,0.510275,-0.891463,-0.115561,0.952916,-0.221254,-0.867646,0.974855,2.441035,-1.711023,0.902400,0.022272,2.097232,0.172188,0.346482,1.162889,-0.557535,-0.896146,0.503553,0.555129,0.129099,0.047571,-0.266444,0.895224,0,0.02,18,0,0,0


In [4]:
%matplotlib inline
from ipywidgets import interact
import seaborn as sns
plt.rcParams['agg.path.chunksize'] = 500

In [5]:
from IPython.display import Image

In [6]:
@interact(Variable = ['XGBoost: Actual % of Fraud', 
                      'XGBoost: Actual Cases of Fraud',
                      'XGBoost: Predicted Likelihood',
                      'XGBoost: False Positives',
                      'XGBoost: False Negatives',
                      'XGBoost: Confusion Matrix',
                      'Random Forest: Actual % of Fraud', 
                      'Random Forest: Actual Cases of Fraud',
                      'Random Forest: Predicted Likelihood',
                      'Random Forest: False Positives',
                      'Random Forest: False Negatives',
                      'Random Forest: Confusion Matrix'
                     ])
def plot(Variable):
    
    if Variable == 'Random Forest: Predicted Likelihood':
        plotdata = data[['predicted_probability', 'proba_decile']].groupby("proba_decile").mean().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.predicted_probability, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Predicted Likelihood of Fraud", labelpad = 15)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax = 1.0, decimals = 0))
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Predicted Fraud Likelihood by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.predicted_probability):
            label = "{:.1f}%" .format(y*100)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
    
    elif Variable == 'Random Forest: Actual % of Fraud':
        plotdata = data[['Class', 'proba_decile']].groupby("proba_decile").mean().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.Class, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Actual % of Fraud", labelpad = 15)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax = 1.0, decimals = 1))
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Actual % of Fraud by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.Class):
            label = "{:.2f}%" .format(y*100)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon

    elif Variable == 'Random Forest: Actual Cases of Fraud':
        plotdata = data[['Class', 'proba_decile']].groupby("proba_decile").sum().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.Class, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Actual Cases of Fraud", labelpad = 15)
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Actual Cases of Fraud by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.Class):
            label = "{:.0f}" .format(y)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
            
    elif Variable == 'Random Forest: False Positives':
        plotdata = data[['FP', 'proba_decile']].groupby("proba_decile").sum().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.FP, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Number of False Positives", labelpad = 15)
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Number of False Positives by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.FP):
            label = "{:.0f}" .format(y)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
            
    elif Variable == 'Random Forest: False Negatives':
        plotdata = data[['FN', 'proba_decile']].groupby("proba_decile").sum().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.FN, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Number of False Negatives", labelpad = 15)
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Number of False Negatives by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.FN):
            label = "{:.0f}" .format(y)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
            
    elif Variable == 'Random Forest: Confusion Matrix':
        display(Image(filename='data/RandomForestCM.png'))
        
    if Variable == 'XGBoost: Predicted Likelihood':
        plotdata = data_XGB[['predicted_probability', 'proba_decile']].groupby("proba_decile").mean().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.predicted_probability, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Predicted Likelihood of Fraud", labelpad = 15)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax = 1.0, decimals = 0))
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Predicted Fraud Likelihood by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.predicted_probability):
            label = "{:.1f}%" .format(y*100)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
    
    elif Variable == 'XGBoost: Actual % of Fraud':
        plotdata = data_XGB[['Class', 'proba_decile']].groupby("proba_decile").mean().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.Class, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Actual % of Fraud", labelpad = 15)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax = 1.0, decimals = 1))
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Actual % of Fraud by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.Class):
            label = "{:.2f}%" .format(y*100)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon

    elif Variable == 'XGBoost: Actual Cases of Fraud':
        plotdata = data_XGB[['Class', 'proba_decile']].groupby("proba_decile").sum().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.Class, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Actual Cases of Fraud", labelpad = 15)
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Actual Cases of Fraud by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.Class):
            label = "{:.0f}" .format(y)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
            
    elif Variable == 'XGBoost: False Positives':
        plotdata = data_XGB[['FP', 'proba_decile']].groupby("proba_decile").sum().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.FP, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Number of False Positives", labelpad = 15)
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Number of False Positives by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.FP):
            label = "{:.0f}" .format(y)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
            
    elif Variable == 'XGBoost: False Negatives':
        plotdata = data_XGB[['FN', 'proba_decile']].groupby("proba_decile").sum().reset_index()
        fig, ax = plt.subplots(figsize=(15,10))
        sns.lineplot(plotdata.proba_decile, plotdata.FN, marker = 'o')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylabel("Number of False Negatives", labelpad = 15)
        ax.set_xlabel("Predicted Probability Vigintile", labelpad = 15)
        ax.set_title("Number of False Negatives by Probability Vigintile", pad = 20)
        for x,y in zip(plotdata.proba_decile, plotdata.FN):
            label = "{:.0f}" .format(y)

            ax.annotate(label, # this is the text
                         (x,y), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(-19,7), # distance from text to points (x,y)
                         ha='center') # horizon
            
    elif Variable == 'XGBoost: Confusion Matrix':
        display(Image(filename='data/XGBoostCM.png'))

interactive(children=(Dropdown(description='Variable', options=('XGBoost: Actual % of Fraud', 'XGBoost: Actual…